## Studenten opdracht

De dataset van het CBS bestaan uit inschrijvingen en diplomauitreikingen van studenten in de periode 2015-2020.
In de originele vraag A ging het over de periode 2008-2015. Omdat deze data niet meer beschikbaar is zal er moeten worden gekozen om de periode 2015-2020 aan te houden.

Opdracht:
- a) Bereken de kans dat een student tot tweemaal toe uitvalt bij een technische studie in de periode 2015-2020
- b) Bereken een belief distribution voor de kans dat een student zowel geslaagde als uitvaller is
- c) Bereken de kans dat een student van Business Studies daarna inschrijft voor een opleiding in het sociale domein, binnen 2 jaar na.



In [1]:
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import Row
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

In [2]:
# Create findspark instance
findspark.init()

# Create spark session
spark = SparkSession.builder.master("local").appName("Linear Regression Model").config("spark.executor.memory", "1gb").getOrCreate()

In [3]:
# Load spark context entry
sc = spark.sparkContext

## Preprocessing
Tijdens deze stap proberen we data van de instroom te combineren met data van de geslaagden over dezelfde periode.
De bedoeling is dat er twee dataframes worden geladen en worden gecombineerd.
Het resultaat is dat we een dataset krijgen waarbij per provincie, gemeente, opleiding en jaar kunnen zien hoeveel instroom en uitstroom er is.

In [4]:
# Loading both text files
data_intake = sc.textFile("04-inschrijvingen-hbo-2020.csv")
data_outgoing = sc.textFile("05-gediplomeerden-hbo-2020.csv")

In [5]:
# Map the columns split by the delimiter=; into a new dataframe
# This changes the RDD from a text file to a 2D array
header_in = data_intake.first()
header_out = data_outgoing.first()

data_intake = data_intake.filter(lambda line: line != header_in).map(lambda line: line.lstrip(";").split(";"))
data_outgoing = data_outgoing.filter(lambda line: line != header_out).map(lambda line: line.lstrip(";").split(";"))

In [6]:
# For both arrays we are going to build a spark DataFrame, which will allow us to do further processing
df_intake = data_intake.map(lambda row: Row(municipality_nr=row[1],
                                            chrono_compartment=row[7],
                                            ed_code_actual=row[9],
                                            ed_name_actual=row[10],
                                            ed_form=row[11],
                                            gender=row[12],
                                            in_2016=row[13],
                                            in_2017=row[14],
                                            in_2018=row[15],
                                            in_2019=row[16],
                                            in_2020=row[17]
                                            )).toDF()

df_outgoing = data_outgoing.map(lambda row: Row(municipality_nr=row[1],
                                            chrono_compartment=row[6],
                                            ed_code_actual=row[8],
                                            ed_name_actual=row[9],
                                            ed_form=row[10],
                                            gender=row[12],
                                            out_2015=row[13],
                                            out_2016=row[14],
                                            out_2017=row[15],
                                            out_2018=row[16],
                                            out_2019=row[17]
                                            )).toDF()

In [7]:
df = df_intake.join(df_outgoing, ["municipality_nr", "chrono_compartment" ,"ed_code_actual", "ed_name_actual", "ed_form", "gender"], how="inner")

In [8]:
df = df.withColumn("in_2016", df["in_2016"].cast(IntegerType())) \
    .withColumn("in_2017", df["in_2017"].cast(IntegerType())) \
    .withColumn("in_2018", df["in_2018"].cast(IntegerType())) \
    .withColumn("in_2019", df["in_2019"].cast(IntegerType())) \
    .withColumn("in_2020", df["in_2020"].cast(IntegerType())) \
    .withColumn("out_2016", df["out_2016"].cast(IntegerType())) \
    .withColumn("out_2017", df["out_2017"].cast(IntegerType())) \
    .withColumn("out_2018", df["out_2018"].cast(IntegerType())) \
    .withColumn("out_2019", df["out_2019"].cast(IntegerType())) \

print(df.count())
df.show(50)

4035
+---------------+--------------------+--------------+--------------------+------------------+------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+
|municipality_nr|  chrono_compartment|ed_code_actual|      ed_name_actual|           ed_form|gender|in_2016|in_2017|in_2018|in_2019|in_2020|out_2015|out_2016|out_2017|out_2018|out_2019|
+---------------+--------------------+--------------+--------------------+------------------+------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+
|           0034|           onderwijs|         44103| M Educational Needs|deeltijd onderwijs|   man|      0|      4|      8|      7|      2|       0|       0|       0|       2|       5|
|           0034|            techniek|         30020|           B HBO-ICT| voltijd onderwijs|   man|    239|    281|    344|    427|    477|      23|      24|      25|      30|      40|
|           0080|  sectoroverstijgend|         49302|M Design Dri

In [9]:
# Now that we have all individual record combined from both sheets, we can eliminate the difference in municipality_nr and gender
# We will look at all cases where chrono_compartment, ed_code_actual, ed_name and ed_form are equal, and sum the total of the in_* and out_* fields
df = df.groupBy([df.chrono_compartment,
                     df.ed_code_actual,
                     df.ed_name_actual]).sum().drop(df.ed_code_actual).drop(df.ed_form)
df.show()

+--------------------+--------------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+
|  chrono_compartment|      ed_name_actual|sum(in_2016)|sum(in_2017)|sum(in_2018)|sum(in_2019)|sum(in_2020)|sum(out_2016)|sum(out_2017)|sum(out_2018)|sum(out_2019)|
+--------------------+--------------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+
|     gezondheidszorg|B Oefentherapie C...|         247|         206|         213|         172|         168|           39|           55|           38|           26|
|           onderwijs|B Opleiding tot l...|          57|          58|          59|          71|          74|           12|           11|           11|           16|
|     gezondheidszorg|M Sport- en Bewee...|           0|           0|          51|          54|          57|            0|            0|           10|            9|
|     taal

### a) Bereken de kans dat een student tot tweemaal toe uitvalt bij een technische studie in de periode 2015-2020

In [10]:
# All we need to do now is to calculate the total incoming and outgoing for each study in the period 2015-2020
# We will create 2 extra columns called in_total and out_total
# Because we have no data for 2020 outgoing, we can only rely on the years where there are both incoming and outdoing students.
# Hence we will drop the year 2020 to get a more accurate estimate of the change
df_sum = df
df_sum = df_sum.withColumn("in_total", sum([df_sum["sum(in_2016)"], df_sum["sum(in_2017)"], df_sum["sum(in_2018)"], df_sum["sum(in_2019)"]]))
df_sum = df_sum.withColumn("out_total", sum([df_sum["sum(out_2016)"], df_sum["sum(out_2017)"], df_sum["sum(out_2018)"], df_sum["sum(out_2019)"]]))
df_sum = df_sum.drop("sum(in_2016)").drop("sum(in_2017)").drop("sum(in_2018)").drop("sum(in_2019)").drop("sum(in_2020)").drop("sum(out_2016)").drop("sum(out_2017)").drop("sum(out_2018)").drop("sum(out_2019)")

df_sum.show()

+--------------------+--------------------+--------+---------+
|  chrono_compartment|      ed_name_actual|in_total|out_total|
+--------------------+--------------------+--------+---------+
|     gezondheidszorg|B Oefentherapie C...|     838|      158|
|           onderwijs|B Opleiding tot l...|     245|       50|
|     gezondheidszorg|M Sport- en Bewee...|     105|       19|
|     taal en cultuur|           M Theater|      69|       32|
|     gezondheidszorg|Ad Management in ...|    1138|      590|
|            economie| B Kunst en Economie|    3129|      541|
|            techniek|  B Ocean Technology|     390|       64|
|     taal en cultuur|M Master of Music...|      37|       19|
|     taal en cultuur|B Interdisciplina...|     164|       40|
|gedrag en maatsch...|      B Sociaal Werk|    3955|      629|
|            techniek|B Maritieme Techniek|    1487|      172|
|            economie|  B Hotel Management|   31661|     5826|
|            techniek|    M Serious Gaming|      64|   

In [11]:
# Now for each row we need to calculate how many students were lost in the time period
# This is not a super accurate measurement since it takes most students more than 4 years to graduate
# But because this is the information we have, we will make it work

df_sum = df_sum.withColumn("dropout", df_sum["in_total"] - df_sum["out_total"])
df_sum = df_sum.withColumn("dropout_ratio", (df_sum["dropout"] / df_sum["in_total"]) * 100)
df_sum.show()

+--------------------+--------------------+--------+---------+-------+------------------+
|  chrono_compartment|      ed_name_actual|in_total|out_total|dropout|     dropout_ratio|
+--------------------+--------------------+--------+---------+-------+------------------+
|     gezondheidszorg|B Oefentherapie C...|     838|      158|    680| 81.14558472553699|
|           onderwijs|B Opleiding tot l...|     245|       50|    195| 79.59183673469387|
|     gezondheidszorg|M Sport- en Bewee...|     105|       19|     86|  81.9047619047619|
|     taal en cultuur|           M Theater|      69|       32|     37| 53.62318840579711|
|     gezondheidszorg|Ad Management in ...|    1138|      590|    548|48.154657293497365|
|            economie| B Kunst en Economie|    3129|      541|   2588|  82.7101310322787|
|            techniek|  B Ocean Technology|     390|       64|    326| 83.58974358974359|
|     taal en cultuur|M Master of Music...|      37|       19|     18| 48.64864864864865|
|     taal

In [12]:
# Now we need to get the total number of dropouts for technical studies and the total number of intake across the board
# We calculate the likelyhood of bering a dropout of a technical study with the folowing formula:
# p = (dropout_technical / sum(in_total))
# Then to calculate the chance that a student is a technical dropout twice by doing
# p_dropout_twice = p ** 2

dropout_technical = df_sum.filter(df_sum.chrono_compartment == "techniek").agg(F.sum("dropout")).collect()[0][0]
print(f"Dropout Technical: {dropout_technical}")
in_total = df_sum.agg(F.sum("in_total")).collect()[0][0]
print(f"Total intake: {in_total}")
p = (dropout_technical / in_total)
print(f"Propability of being a technical dropout: {p}")
p_dropout_twice = p ** 2
print(f"Propability of being a technical dropout twice: {p_dropout_twice}")

Dropout Technical: 322902
Total intake: 1982797
Propability of being a technical dropout: 0.16285176949531394
Propability of being a technical dropout twice: 0.026520698827754863


### b) Bereken een belief distribution voor de kans dat een student zowel geslaagde als uitvaller is

In [13]:
# We do this my calculating the chance that a student has graduated
# Then we calculate the chance that a student is a dropout
# Then we multiply the chance of graduation with the chance of being a dropout

total_dropout = df_sum.agg(F.sum("dropout")).collect()[0][0]
print(f"Total dropouts: {total_dropout}")
p_dropout = total_dropout / in_total
print(f"P of being a dropout: {p_dropout}")
total_graduated = df_sum.agg(F.sum("out_total")).collect()[0][0]
print(f"Total graduates: {total_graduated}")
p_graduated = total_graduated / in_total
print(f"P of being graduated: {p_graduated}")
p = p_graduated * p_dropout
print(f"P of being both graduated and a dropout: {p}")


Total dropouts: 1656918
P of being a dropout: 0.8356468160885859
Total graduates: 325879
P of being graduated: 0.16435318391141404
P of being both graduated and a dropout: 0.13734121484959494


### c) Bereken de kans dat een student van Business Studies daarna inschrijft voor een opleiding in het sociale domein, binnen 2 jaar na

In [14]:
df = df.withColumnRenamed("sum(in_2016)", "in_2016") \
        .withColumnRenamed("sum(in_2017)", "in_2017") \
        .withColumnRenamed("sum(in_2018)", "in_2018") \
        .withColumnRenamed("sum(in_2019)", "in_2019") \
        .withColumnRenamed("sum(in_2020)", "in_2020") \
        .withColumnRenamed("sum(out_2016)", "out_2016") \
        .withColumnRenamed("sum(out_2017)", "out_2017") \
        .withColumnRenamed("sum(out_2018)", "out_2018") \
        .withColumnRenamed("sum(out_2019)", "out_2019")

In [15]:
# With the original df we will look at the chance that a graduate from business studies enrolls for a social study within 2 years of graduation
# We need to know how many "B Business Studies" graduates there are for each year
# Then we need to calculate what the total number of intake is for the social studies ("gedrag en maatschappij") for the next year and the year after that
# Then we calculate the chance that the business studies graduate enrolls in the total intake of the next 2 years
# We repeat this process for the business studies graduates years 2016, 2017 and 2018
# This way we still know what the chance is that the student enrolls in either 2019 or 2020

bs_grads = df.filter(df["ed_name_actual"] == "B Business Studies")

bs_grad_2016 = bs_grads.agg(F.sum("out_2016")).collect()[0][0]
print(f"Total business studies graduates in 2016: {bs_grad_2016}")
bs_grad_2017 = bs_grads.agg(F.sum("out_2017")).collect()[0][0]
print(f"Total business studies graduates in 2017: {bs_grad_2017}")
bs_grad_2018 = bs_grads.agg(F.sum("out_2018")).collect()[0][0]
print(f"Total business studies graduates in 2018: {bs_grad_2018}")

in_social = df.filter(df["chrono_compartment"] == "gedrag en maatschappij")

in_social_2017 = in_social.agg(F.sum("in_2017")).collect()[0][0]
print(f"Total social intake 2017: {in_social_2017}")
in_social_2018 = in_social.agg(F.sum("in_2018")).collect()[0][0]
print(f"Total social intake 2018: {in_social_2018}")
in_social_2019 = in_social.agg(F.sum("in_2019")).collect()[0][0]
print(f"Total social intake 2019: {in_social_2019}")
in_social_2020 = in_social.agg(F.sum("in_2020")).collect()[0][0]
print(f"Total social intake 2020: {in_social_2020}")

Total business studies graduates in 2016: 536
Total business studies graduates in 2017: 904
Total business studies graduates in 2018: 416
Total social intake 2017: 67564
Total social intake 2018: 65019
Total social intake 2019: 64794
Total social intake 2020: 67976


In [16]:
# The dataset contains no data on what happends with graduates after they graduate
# This means that we simply cannot know which graduated students enrolled again in the social domain
# We can however, calculate the chance that any given graduate is also enrolled in a social study in the next year and the year after that

p_reenrolled_2016_2018 = bs_grad_2016 / (in_social_2017 + in_social_2018)
p_reenrolled_2017_2019 = bs_grad_2017 / (in_social_2018 + in_social_2019)
p_reenrolled_2018_2020 = bs_grad_2018 / (in_social_2019 + in_social_2020)
print(f"P re-enrollment in 2016 - 2018: {p_reenrolled_2016_2018}")
print(f"P re-enrollment in 2017 - 2019: {p_reenrolled_2017_2019}")
print(f"P re-enrollment in 2018 - 2020: {p_reenrolled_2018_2020}")

P re-enrollment in 2016 - 2018: 0.004042750578882662
P re-enrollment in 2017 - 2019: 0.006963863403511204
P re-enrollment in 2018 - 2020: 0.0031332379302553286


In [17]:
# Lastly we can calculate the average change over the entire period
p_total = p_reenrolled_2016_2018 + p_reenrolled_2017_2019 + p_reenrolled_2018_2020
print(f"Sum P of re-enrollment in period: {p_total}")
p_avg = p_total / 3
print(f"Avg P of re-enrollment in period: {p_avg}")

Sum P of re-enrollment in period: 0.014139851912649194
Avg P of re-enrollment in period: 0.0047132839708830645


This means that the total chance of a given graduate from business studies being re-enrolled in social studies over the entire time period is
0.014139851912649194

The average propability of all the time periods is 0.0047132839708830645

In [18]:
spark.stop()